In [0]:
from __future__ import absolute_import, division, print_function
%tensorflow_version 2.x
import tensorflow as tf

In [2]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported)
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [3]:
# Each tf.Tensor has shape and a datatype
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


In [0]:
# Tensor can be backed by accelerator memory (like GPU, TPU)
# Tensors are immutable

# Tensorflow operations automatically convert NumPy ndarrays to Tensors.
# NumPy operation automatically convert Tensors to NumPy ndarrays.

# Tensors are explicitly converted to NumPy ndarrays using their .numpy() method


In [5]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


These conversion are typically cheap since the array and tf.Tensor share the underlying memory representation, if possible. However, sharing the underlying representation isn't always possible since the tf.Tensor may be hosted GPU memory while Numpy arrays are always backed by host memory, and the conversion involves a copy from GPU to host memory.

In [7]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available")
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0: "),
print(x.device.endswith('GPU:0'))

# Withotu any annotations, TensorFlow automatically decides whether to use
# the GPU or CPU for an operation-copying the tensor between CPU and GPU memory,
# if necessary.
# Tensors produced by an operation are typically backed by the memory of the decide
# on which the operation executed.

Is there a GPU available
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0: 
True


In [8]:
# The Tensor.device property provides a fully qualified string name of the device
# hosting the contents of the tensor.
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [0]:
# Explicit Device placement
# In TensorFlow, placement refers to how individual operations are assigned
# (placed on) a device for execution

import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time() - start

  print("10 loops: {:0.2f}ms".format(1000*result))

In [11]:
# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

On CPU:
10 loops: 408.70ms


In [13]:
# Force execution on GPU#0 if available
if tf.config.experimental.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert  x.device.endswith("GPU:0")
    time_matmul(x)

On GPU:
10 loops: 9.35ms


In [0]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CVS file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
          Line 2
          Line 3
          """)
ds_file = tf.data.TextLineDataset(filename)

In [0]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [20]:
print("Elements of ds_tensors")
for x in ds_tensors:
  print(x)

Elements of ds_tensors
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([16  9], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)


In [21]:
print('\nElements in ds_file:')
for x in ds_file:
  print(x)


Elements in ds_file:
tf.Tensor([b'Line 1' b'          Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'          Line 3' b'          '], shape=(2,), dtype=string)
